In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import tensorflow as tf
!pip install tensorflow-addons

In [3]:
!pip install tensorflow-addons


In [4]:
import tensorflow_addons as tfa
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
from keras.utils import to_categorical
import os
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Reshape, Activation, Dot
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization, LayerNormalization, Normalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [5]:
train_folder = "../archive/Train"
test_folder = "../archive/Test"

train_datagen = ImageDataGenerator(
    validation_split = 0.3,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode="nearest",
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=False,
    vertical_flip=False
)
test_datagen = ImageDataGenerator()

In [6]:
# Label Overview
classes = { 
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)', 
    2:'Speed limit (50km/h)', 
    3:'Speed limit (60km/h)', 
    4:'Speed limit (70km/h)', 
    5:'Speed limit (80km/h)', 
    6:'End of speed limit (80km/h)', 
    7:'Speed limit (100km/h)', 
    8:'Speed limit (120km/h)', 
    9:'No passing', 
    10:'No passing veh over 3.5 tons', 
    11:'Right-of-way at intersection', 
    12:'Priority road', 
    13:'Yield', 
    14:'Stop', 
    15:'No vehicles', 
    16:'Veh > 3.5 tons prohibited', 
    17:'No entry', 
    18:'General caution', 
    19:'Dangerous curve left', 
    20:'Dangerous curve right', 
    21:'Double curve', 
    22:'Bumpy road', 
    23:'Slippery road', 
    24:'Road narrows on the right', 
    25:'Road work', 
    26:'Traffic signals', 
    27:'Pedestrians', 
    28:'Children crossing', 
    29:'Bicycles crossing', 
    30:'Beware of ice/snow',
    31:'Wild animals crossing', 
    32:'End speed + passing limits', 
    33:'Turn right ahead', 
    34:'Turn left ahead', 
    35:'Ahead only', 
    36:'Go straight or right', 
    37:'Go straight or left', 
    38:'Keep right', 
    39:'Keep left', 
    40:'Roundabout mandatory', 
    41:'End of no passing', 
    42:'End no passing veh > 3.5 tons'
}

In [7]:
num_classes = len(classes)
target_size_x = 48
target_size_y = 48
seed = 123
batch_size = 32
np.random.seed(seed)
tf.random.set_seed(seed)

In [8]:
import pandas as pd
df=pd.read_csv("../archive/Train.csv")
df.head()

,Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path
0,27,26,5,5,22,20,20,Train/20/00020_00000_00000.png
1,28,27,5,6,23,22,20,Train/20/00020_00000_00001.png
2,29,26,6,5,24,21,20,Train/20/00020_00000_00002.png
3,28,27,5,6,23,22,20,Train/20/00020_00000_00003.png
4,28,26,5,5,23,21,20,Train/20/00020_00000_00004.png


In [9]:
x_train = []
y_train = []

for i in tqdm(range(len(classes))):         
    for imgFile in os.listdir(os.path.join(train_folder, str(i))):
        img_path = os.path.join(os.path.join(train_folder, str(i)), imgFile)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (target_size_x,target_size_y))

        x_train.append(image/255.0)
        y_train.append(i)

100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [16:09<00:00, 22.55s/it]


In [10]:
x_train = np.array(x_train, dtype = 'float32')
y_train = np.array(y_train, dtype = 'int32')
y_train = keras.utils.to_categorical(y_train, num_classes=43)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)
print("x_train.shape", x_train.shape)
print("x_valid.shape", x_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

x_train.shape (31367, 48, 48, 3)
x_valid.shape (7842, 48, 48, 3)
y_train.shape (31367, 43)
y_valid.shape (7842, 43)


In [11]:
train_flow = train_datagen.flow(
    x_train, y_train,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)
val_flow = test_datagen.flow(
    x_val, y_val,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

In [12]:
class LRN2D(tf.keras.layers.Layer):
    """
    Local Contrast Normalisation
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__()
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = X.shape
        half_n = self.n // 2
        input_sqr = T.sqr(X)
        extra_channels = T.alloc(0., b, ch + 2*half_n, r, c)
        input_sqr = T.set_subtensor(extra_channels[:, half_n:half_n+ch, :, :], input_sqr)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i+ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        return {"name": self.__class__.__name__,
                "alpha": self.alpha,
                "k": self.k,
                "beta": self.beta,
                "n": self.n}

In [13]:
def creat_cnn_model():
    inputs = layers.Input((target_size_x, target_size_y, 3))
    
    norm = Normalization(mean=[0.4914, 0.4822, 0.4465], 
                  variance=[np.square(0.247), 
                            np.square(0.243), 
                            np.square(0.261)])(inputs)
    
    x = Conv2D(filters=200, kernel_size=(7,7), padding='valid', activation='relu', kernel_initializer="he_normal")(norm)
    x = layers.MaxPool2D(pool_size=(2, 2))(x)
    x = LRN2D()(x)
    
    x = Conv2D(filters=250, kernel_size=(4,4), padding='valid', activation='relu', kernel_initializer="he_normal")(x)
    x = layers.MaxPool2D(pool_size=(2, 2))(x)
    x = LRN2D()(x)
    x = Dropout(rate=0.25)(x)
    
    x = Conv2D(filters=350, kernel_size=(4,4), padding='valid', activation='relu', kernel_initializer="he_normal")(x)
    x = layers.MaxPool2D(pool_size=(2, 2))(x)
    x = LRN2D()(x)
    x = Dropout(rate=0.25)(x)
    
    x = Flatten()(x)
    x = Dense(400, activation='relu')(x)
    x = Dropout(rate=0.5)(x)
    classifier = Dense(43, activation='softmax')(x)
    return keras.Model(inputs=inputs, outputs=classifier)

In [14]:
model = creat_cnn_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 normalization (Normalizati  (None, 48, 48, 3)         0         
 on)                                                             
                                                                 
 conv2d (Conv2D)             (None, 42, 42, 200)       29600     
                                                                 
 max_pooling2d (MaxPooling2  (None, 21, 21, 200)       0         
 D)                                                              
                                                                 
 lrn2d (LRN2D)               (None, 21, 21, 200)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 18, 18, 250)       800250

In [15]:
class CosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, min_lr, max_lr, warmup_steps=4000, alpha=0):
        super(CosineDecay, self).__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.warmup_steps = warmup_steps
        self.max_steps = warmup_steps * 10
        self.alpha = alpha

    def __call__(self, step):
        arg1 = (self.max_lr - self.min_lr) * step / self.warmup_steps + self.min_lr
        min_step = tf.math.minimum(step, self.max_steps)
        cosine_decay = 0.5 * (1 + tf.math.cos(np.pi * min_step / self.max_steps))
        decayed = (1 - self.alpha) * cosine_decay + self.alpha
        arg2 = (self.max_lr - self.min_lr) * decayed + self.min_lr
        return tf.math.minimum(arg1, arg2)
    
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        print("\nEpoch %05d: Current learning rate is %6.4f." % (epoch, lr))

In [ ]:
epochs = 30   # number of all epochs in training
stop_patience = 10   # number of epochs to wait before stopping training if monitored value does not improve
batches = int(len(x_train) / batch_size)  # number of training batch to run per epoch

warmup_steps = batches * 5
learning_rate = CosineDecay(
    min_lr=1E-5, max_lr=1E-2, warmup_steps=warmup_steps
)
    
checkpoint = ModelCheckpoint(
      filepath='cnn48-{epoch:02d}-{val_acc:.4f}.h5',
      save_weights_only=True,
      monitor='val_acc',
      mode='max',
      save_best_only=True,
      initial_value_threshold=0.995)

earlystop = EarlyStopping(
      monitor='val_loss', patience=stop_patience, mode='min', restore_best_weights=True)

loss_fn = tf.keras.losses.CategoricalCrossentropy()

optimizer = keras.optimizers.SGD(learning_rate=0.02)


model.compile(
        optimizer, loss=loss_fn,
        metrics=[
        keras.metrics.CategoricalAccuracy(name="acc"),
        keras.metrics.TopKCategoricalAccuracy(5, name="top-5-accuracy"),
    ],
)
history = model.fit(
    x = train_flow,
    batch_size=batch_size,
    validation_data=val_flow,
    steps_per_epoch=batches,
    verbose=1,
    epochs=epochs,
    callbacks=[checkpoint, earlystop],
    validation_steps= None,
    shuffle= False,
)

Epoch 1/30
980/980 [==============================] - 3009s 3s/step - loss: 2.3209 - acc: 0.3572 - top-5-accuracy: 0.6757 - val_loss: 0.8504 - val_acc: 0.7431 - val_top-5-accuracy: 0.9540
Epoch 2/30
980/980 [==============================] - 2530s 3s/step - loss: 0.9946 - acc: 0.6923 - top-5-accuracy: 0.9383 - val_loss: 0.2224 - val_acc: 0.9397 - val_top-5-accuracy: 0.9971
Epoch 3/30
980/980 [==============================] - 2501s 3s/step - loss: 0.5426 - acc: 0.8282 - top-5-accuracy: 0.9803 - val_loss: 0.1016 - val_acc: 0.9745 - val_top-5-accuracy: 0.9994
Epoch 4/30
584/980 [================>.............] - ETA: 18:36 - loss: 0.3798 - acc: 0.8801 - top-5-accuracy: 0.9881